In [10]:
%matplotlib inline
import matplotlib.pyplot as plt

from scipy.integrate import odeint
import py_analyze_tools.tools as pat
import numpy as np
from pprint import pprint
import math

In [2]:
traj = pat.Trajectory("../generate/simple_trajectory_.h5")
traj.counts = traj.counts[:45*500]
traj.add_fusion(0, 1, 2)
traj.add_fission(2, 0, 1)
traj.add_conversion(0, 1)
traj.add_conversion(0, 3)
traj.add_conversion(3, 0)
xi = traj.estimate(.003)

max counts = 6000.0, min nonzero counts = 1.0


In [20]:
def magnitude(x):
    return math.floor(math.log10(x))

In [21]:
cv = pat.CV(traj)
result = cv.calculate_cost([0], range(0, 6000), range(6000, traj.n_time_steps))
norm_of_coeff = np.linalg.norm(result.coefficients[0], ord=1)
print("norm of coefficients for alpha=0: {}".format(norm_of_coeff))
quotient = result.costs_test[0]/norm_of_coeff
print("quotient = {}, order of magnitude = {}".format(quotient, magnitude(quotient)))

norm of coefficients for alpha=0: 0.012067573144616576
quotient = 0.220080932430265, order of magnitude = -1


In [29]:
search_grid = np.linspace(0, 10**(magnitude(quotient)+1), num=500)

In [ ]:
cv_result = cv.calculate_cost(search_grid, range(0, 6000), range(6000, traj.n_time_steps))
min_idx = np.argmin(cv_result.costs_test)
print("best suited alpha found at idx={}, alpha={}, costs_test={}".format(min_idx, cv_result.alphas[min_idx], cv_result.costs_test[min_idx]))